## Loading the libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

## Load the data

In [2]:
data = pd.read_csv('train.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Create a Column Transformer to be applied on the data

- To create a column transformer
1. To create a pipeline for numerical varialbes
2. To create a pipeline for categorical variables
3. To create the column transformer for the above 2 pipelines

## 1. Create a numerical variable pipeline

- Steps to create pipeline
    1. To import all the necessary libraries for preprocessing num variables
    2. Define the steps
    3. create a pipeline using the Pipeline() from sklearn library

In [3]:
from sklearn.impute import SimpleImputer
si_num = SimpleImputer(strategy = 'mean')
si_num

SimpleImputer()

In [4]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
mms

MinMaxScaler()

In [5]:
## Using the shortcut method to create the num_pipeline

from sklearn.pipeline import make_pipeline
num_pipe = make_pipeline(si_num, mms)
num_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('minmaxscaler', MinMaxScaler())])

## Create a pipeline for Categorical Variables

- Steps to create pipeline
    1. To import all the necessary libraries for preprocessing categorical variables
    2. Define the steps
    3. create a pipeline using the Pipeline() from sklearn library

In [6]:
from sklearn.impute import SimpleImputer
si_cat = SimpleImputer(strategy = 'most_frequent')
si_cat

SimpleImputer(strategy='most_frequent')

In [7]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
ohe

OneHotEncoder()

In [8]:
## Using the shortcut method for create the cat_pipe

from sklearn.pipeline import make_pipeline
cat_pipe = make_pipeline(si_cat, ohe)
cat_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('onehotencoder', OneHotEncoder())])

## Create the column transformer using num_pipe and cat_pipe
- To create column transformer 
    1. Seperate all num and categorical variables from the data
    2. Create the column transformer using the sklearn.compose library
    3. apply the transformer on the data.

In [9]:
num_features = ['Age', 'Fare']
cat_features = ['Sex', 'Embarked']

all_features = num_features + cat_features
all_features

['Age', 'Fare', 'Sex', 'Embarked']

In [10]:
from sklearn.compose import ColumnTransformer
col_trans = ColumnTransformer(
                                    [('num_pipeline', num_pipe, num_features), 
                                    ('cat_pipeline', cat_pipe, cat_features)]
)
col_trans

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('minmaxscaler',
                                                  MinMaxScaler())]),
                                 ['Age', 'Fare']),
                                ('cat_pipeline',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder())]),
                                 ['Sex', 'Embarked'])])

In [11]:
data_piped = col_trans.fit_transform(data)
data_piped

array([[0.27117366, 0.01415106, 0.        , ..., 0.        , 0.        ,
        1.        ],
       [0.4722292 , 0.13913574, 1.        , ..., 1.        , 0.        ,
        0.        ],
       [0.32143755, 0.01546857, 1.        , ..., 0.        , 0.        ,
        1.        ],
       ...,
       [0.36792055, 0.04577135, 1.        , ..., 0.        , 0.        ,
        1.        ],
       [0.32143755, 0.0585561 , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.39683338, 0.01512699, 0.        , ..., 0.        , 1.        ,
        0.        ]])

In [12]:
data_test = pd.read_csv('test.csv')
data_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [13]:
data_test_piped = col_trans.fit_transform(data_test)
data_test_piped = pd.DataFrame(data_test_piped, columns = ['Age', 'Fare', 'Female', 'Male', 'C', 'Q', 'S'])
data_test_piped

,Age,Fare,Female,Male,C,Q,S
0,0.452723,0.015282,0.0,1.0,0.0,1.0,0.0
1,0.617566,0.013663,1.0,0.0,0.0,0.0,1.0
2,0.815377,0.018909,0.0,1.0,0.0,1.0,0.0
3,0.353818,0.016908,0.0,1.0,0.0,0.0,1.0
4,0.287881,0.023984,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...
413,0.396975,0.015713,0.0,1.0,0.0,0.0,1.0
414,0.512066,0.212559,1.0,0.0,1.0,0.0,0.0
415,0.505473,0.014151,0.0,1.0,0.0,0.0,1.0
416,0.396975,0.015713,0.0,1.0,0.0,0.0,1.0


In [14]:
data_test = pd.concat([data_test, data_test_piped], axis = 1)
data_test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age,Fare,Female,Male,C,Q,S
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0.452723,0.015282,0.0,1.0,0.0,1.0,0.0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,0.617566,0.013663,1.0,0.0,0.0,0.0,1.0
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0.815377,0.018909,0.0,1.0,0.0,1.0,0.0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0.353818,0.016908,0.0,1.0,0.0,0.0,1.0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,0.287881,0.023984,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,0.396975,0.015713,0.0,1.0,0.0,0.0,1.0
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,0.512066,0.212559,1.0,0.0,1.0,0.0,0.0
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,0.505473,0.014151,0.0,1.0,0.0,0.0,1.0
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S,0.396975,0.015713,0.0,1.0,0.0,0.0,1.0
